## --- code --

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data_file_path = r'..\data'
data_file_name = 'output_data_loader_copy1.xlsx'
tm_file_path = r'..\data'
tm_file_name = 'tag-mapping-steadystate.xlsx'
ss_sheetname = 'ss_det'

In [3]:
# Load data
fullfile_data = os.path.join(data_file_path, data_file_name)
fullfile_tm = os.path.join(tm_file_path, tm_file_name)
df_data = pd.read_excel(fullfile_data, index_col=[0])
df_tm = pd.read_excel(fullfile_tm, sheet_name=ss_sheetname)
df_tm = df_tm[df_tm['Tag Name-SS'].notna()].reset_index(drop=True)

In [4]:
# df_tm['STDDEV'].iloc[0]

In [5]:
duration_hours = 4
sampling_interval_min = 1
steady_state_rows = [0, 1, 2, 3, 4]   
not_steady_state_rows = []  
missing_data_rows = [0, 4] 
per_to_delete = 20

In [6]:
td = pd.Timedelta(minutes=sampling_interval_min)
num_columns = duration_hours * 60 // sampling_interval_min
start_date = pd.Timestamp('2024-08-01 01:00:00')
time_series = pd.date_range(start=start_date, periods=num_columns, freq=td).strftime('%Y-%m-%d %H:%M:00')
#df_tm.set_index('Tag Name-SS', inplace=True)
fake_data = pd.DataFrame(index=df_tm['Tag Name-SS'], columns=time_series)

In [7]:
# Generate fake data based on user-defined row types
for index, row in df_tm.iterrows():
    if index in steady_state_rows:
        marg = 0.75
    else:
        marg = 1.5
        
    pct_50 = 10000
    sigma = df_tm['STDDEV'].iloc[index]  #0.01 if index in steady_state_rows else 0.05
    THR = df_tm['THR'].iloc[index] #2 if index in steady_state_rows else 3
    pct_99 = sigma * THR * pct_50 * marg + pct_50
    pct_1 = pct_50 - (pct_99 - pct_50)
    tag_data = np.random.uniform(low=pct_1, high=pct_99, size=num_columns)
    fake_data.loc[row['Tag Name-SS']] = tag_data

for index, row in df_tm.iterrows():
    if index in missing_data_rows:
        num_cols = fake_data.shape[1]
        #per_to_delete=40
        cells_to_delete = int((per_to_delete / 100) * num_cols)
        cols_to_delete = np.random.choice(len(fake_data.columns), cells_to_delete, replace=False)
        for icol in cols_to_delete:
            fake_data.iat[index, icol] = np.nan

output_path = os.path.join(data_file_path, 'df_fake_data.xlsx')
fake_data.to_excel(output_path)
print("Fake plant data has been saved successfully.")

Fake plant data has been saved successfully.
